In [1]:
import pandas as pd
from datetime import datetime, timedelta

# Ruta base
base = "/lustre/home/mvalenzuela/Ocotillo/DataAqua/"

# === 1. Cargar REMAS ===
remas = pd.read_csv(base + "Datos/Eto_Cajeme_Remas_2022_2023.csv")
remas["fecha"] = pd.to_datetime(remas["Fecha"])
remas = remas[["fecha", "Eto (REMAS)"]].rename(columns={"Eto (REMAS)": "Eto_REMAS"})

# === 2. Cargar UAMEX (.xlsx) ===
def cargar_uamex_xlsx(archivo, año_ini):
    df = pd.read_excel(archivo)
    df["fecha"] = df["Día"].apply(lambda d: datetime(año_ini if d >= 300 else año_ini + 1, 1, 1) + timedelta(days=d - 1))
    return df[["fecha", "ET0"]].rename(columns={"ET0": "Eto_UAMEX"})

ruta_uamex = "Datos/ET Calculada - Por Periodo de Cultivo - 2010-2024/Cajeme/"
uamex1 = cargar_uamex_xlsx(base + ruta_uamex + "2021-2022-ET_CALCULADA_CAJEME.xlsx", 2021)
uamex2 = cargar_uamex_xlsx(base + ruta_uamex + "2022-2023-ET_CALCULADA_CAJEME.xlsx", 2022)
uamex = pd.concat([uamex1, uamex2])

# === 3. Cargar UNISON (.csv con columnas Año_(YEAR), Día, ET0/ETO) ===
def cargar_unison(archivo):
    df = pd.read_csv(archivo)
    
    # Normaliza nombres de columnas para hacer búsqueda flexible
    columnas = {c.lower().strip(): c for c in df.columns}
    
    col_year = next((v for k, v in columnas.items() if "año" in k or "year" in k), None)
    col_doy  = next((v for k, v in columnas.items() if "día" in k or "doy" in k), None)
    col_et0  = next((v for k, v in columnas.items() if "et0" in k or "eto" in k), None)

    if not all([col_year, col_doy, col_et0]):
        raise ValueError(f"No se encontraron columnas necesarias en {archivo}")

    df["fecha"] = df.apply(lambda row: datetime(int(row[col_year]), 1, 1) + timedelta(days=int(row[col_doy]) - 1), axis=1)
    return df[["fecha", col_et0]].rename(columns={col_et0: "Eto_UNISON"})

ruta_unison = "Salidas_ETo11/Periodo de Cultivo ETo/Cajeme/"
unison1 = cargar_unison(base + ruta_unison + "Cajeme-FAO56-2021-2022-SALIDA.csv")
unison2 = cargar_unison(base + ruta_unison + "Cajeme-FAO56-2022-2023-SALIDA.csv")
unison = pd.concat([unison1, unison2])

# === 4. Unir por fecha ===
df = remas.merge(uamex, on="fecha", how="inner").merge(unison, on="fecha", how="inner")
df.insert(0, "Region", "Cajeme")
df = df[["Region", "fecha", "Eto_REMAS", "Eto_UAMEX", "Eto_UNISON"]]

# === 5. Exportar ===
df.to_csv("comparacion_4_eto.csv", index=False)



In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# Leer datos
df = pd.read_csv("comparacion_4_eto.csv", parse_dates=["fecha"])
df = df.sort_values("fecha")

# Crear PDF
with PdfPages("comparacion_eto_por_periodo_.pdf") as pdf:

    # Definir periodos personalizados
    periodos = [
        ("2022 (Ene-Abr)", (pd.Timestamp("2022-01-01"), pd.Timestamp("2022-04-30"))),
        ("2022 (Oct-Dic)", (pd.Timestamp("2022-10-01"), pd.Timestamp("2022-12-31"))),
    ]

    # Agregar también los demás años automáticamente (excepto 2022 que ya lo dividimos)
    for year in df["fecha"].dt.year.unique():
        if year != 2022:
            periodos.append((f"{year}", (pd.Timestamp(f"{year}-01-01"), pd.Timestamp(f"{year}-12-31"))))

    # Generar gráficas
    for nombre, (inicio, fin) in periodos:
        subset = df[(df["fecha"] >= inicio) & (df["fecha"] <= fin)]
        if subset.empty:
            continue

        # Calcular rango con más margen (20%)
        min_eto = subset[["Eto_REMAS", "Eto_UAMEX", "Eto_UNISON"]].min().min()
        max_eto = subset[["Eto_REMAS", "Eto_UAMEX", "Eto_UNISON"]].max().max()
        y_margin = (max_eto - min_eto) * 0.2

        # Plot
        plt.figure(figsize=(12, 6))
        plt.plot(subset["fecha"], subset["Eto_REMAS"], label="REMAS", color="blue", linewidth=1.8)
        plt.plot(subset["fecha"], subset["Eto_UAMEX"], label="UAMEX", color="green", linewidth=1.8)
        plt.plot(subset["fecha"], subset["Eto_UNISON"], label="UNISON", color="red", linewidth=1.8)
        plt.title(f"ET0 - Cajeme ({nombre})", fontsize=14)
        plt.xlabel("Fecha", fontsize=12)
        plt.ylabel("ET0 (mm/día)", fontsize=12)
        plt.ylim(min_eto - y_margin, max_eto + y_margin)
        plt.xticks(rotation=45)
        plt.grid(True, linestyle="--", alpha=0.6)
        plt.legend()
        plt.tight_layout()
        pdf.savefig()
        plt.close()
